In [2]:
import tensorflow as tf
from glob import glob
import os
import cv2
from PIL import Image
from tensorflow.keras.utils import normalize
import numpy as np



In [3]:
img_dataset = []
msk_dataset = []

f_img = sorted(glob('dataset/train/asagao/*'))
for f in f_img:
    image = cv2.imread(f)
    # image = image.resize((SIZE, SIZE))
    #Nomalize an image from 0-255 to 0 - 1
    image = image.astype('float32') / 255.0
    img_dataset.append(np.array(image))

f_msk = sorted(glob('dataset/validation/asagao/*'))
for f in f_msk:
    image = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    (thresh, im_bw) = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    image = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY)[1]
    # image = Image.fromarray(image)
    # image = image.resize((SIZE, SIZE))
    msk_dataset.append(np.array(image))

# img_dataset = (img_dataset - v_min)/(v_max - v_min)
# print(img_dataset.shape)
# #Normalize images
# img_dataset = normalize(np.array(img_dataset), axis=1) #wrong normalization




In [4]:
asagao_dataset = np.array(img_dataset)
# img_dataset = np.array(img_dataset)
# #D not normalize masks, just rescale to 0 to 1.a
asagao_msk_dataset = np.expand_dims(np.array(msk_dataset) /255., -1)

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
X_train, X_test, y_train, y_test = train_test_split(asagao_dataset, asagao_msk_dataset, test_size= 0.10, random_state=0)
# y_train = to_categorical(y_train, 3)

### DATA SANITY CHECK

In [ ]:
# import random
# import numpy as np

# %matplotlib inline

# import matplotlib.pyplot as plt
# X_train_quick_test, X_test_quick_test, y_train_quick_test, y_test_quick_test = train_test_split(X_train, y_train, test_size = 0.9, random_state = 0)
# image_number = random.randint(0, len(X_train_quick_test))
# plt.figure(figsize=(12, 6))
# plt.subplot(121)
# plt.imshow(np.reshape(X_train_quick_test[image_number], (256, 256,3)))
# plt.subplot(122)
# plt.imshow(np.reshape(y_train_quick_test[image_number], (256, 256,1)), cmap='gray')
# plt.show()

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout,Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
    
# model.summary()
input_w = 256
input_h = 256
input_ch = 3

s  = Input((input_w, input_h, input_ch))
#Downscale path -----------------------------------------------------------------------------------
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = Dropout(0.1)(c1)
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = MaxPooling2D((2, 2))(c1)
#100
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = Dropout(0.1)(c2)
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = MaxPooling2D((2, 2))(c2)
#50
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = Dropout(0.2)(c3)
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = MaxPooling2D((2, 2))(c3)
#15
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = Dropout(0.2)(c4)
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = Dropout(0.3)(c5)
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Scale up path ----------------------------------------------------------------------------------
u6 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = Dropout(0.2)(c6)
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
    
u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = Dropout(0.2)(c7)
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
    
u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = Dropout(0.1)(c8)
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
    
u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = Dropout(0.1)(c9)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
model = Model(inputs=[s], outputs=[outputs])
opt = Adam(learning_rate = 0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 256, 256, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
dropout_9 (Dropout)             (None, 256, 256, 16) 0           conv2d_19[0][0]                  
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 256, 256, 16) 2320        dropout_9[0][0]                  
____________________________________________________________________________________________

### train with asagao pixel

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', verbose = 1)
mc  = ModelCheckpoint("50-30mUnet.h5", monitor = "val_loss", save_best_only = True)

callback_list = [es, mc]


In [7]:
# history_previous = history
history = model.fit(X_train, 
                    y_train,
                    verbose = 1,
                    batch_size = 32,
                    epochs = 100,
                    validation_data = (X_test, y_test),
                    shuffle = False)
model.save("50m_30m_Unet")



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

history = history_previous

acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'b', label='Training acc',linewidth=2)
plt.plot(epochs, val_acc, 'r--', label='Validation acc',linewidth=2)
plt.title('Training  accuracy')
plt.legend()

plt.subplot(1, 2, 2)

plt.plot(epochs, loss, 'b', label='Training loss')

plt.plot(epochs, val_loss, 'k', label='Validationloss ')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
##################################
#IOU
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.6

intersection = np.logical_and(y_test, y_pred_thresholded)
union = np.logical_or(y_test, y_pred_thresholded)
iou_score = np.sum(intersection) / np.sum(union)
print("IoU socre is: ", iou_score)

### train with non-asagao


In [ ]:
# non_asagao_dataset = []
# non_asagao_msk_dataset = []

# f_img = sorted(glob('dataset/non_asagao_train/*'))
# for f in f_img:
#     image = cv2.imread(f)
#     # image = image.resize((SIZE, SIZE))
#     #Nomalize an image from 0-255 to 0 - 1
#     image = image.astype('float32') / 255.0
#     img_dataset.append(np.array(image))

# f_msk = sorted(glob('dataset/non_asagao_validation/*'))
# for f in f_msk:
#     image = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
#     (thresh, im_bw) = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#     image = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY)[1]
#     # image = Image.fromarray(image)
#     # image = image.resize((SIZE, SIZE))
#     msk_dataset.append(np.array(image))


In [ ]:
# non_asagao_dataset = np.array(img_dataset)
# # img_dataset = np.array(img_dataset)
# # #D not normalize masks, just rescale to 0 to 1.a
# non_asagao_msk_dataset = np.expand_dims(np.array(msk_dataset) /255., -1)

In [ ]:
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.utils import to_categorical
# X_train, X_test, y_train, y_test = train_test_split(non_asagao_dataset, non_asagao_msk_dataset, test_size= 0.20, random_state=0)
# y_train = to_categorical(y_train, 3)

In [ ]:
# history = model.fit(X_train, 
#                     y_train,
#                     verbose = 1,
#                     batch_size = 16,
#                     epochs = 50,
#                     validation_data = (X_test, y_test),
#                     shuffle = False)
# model.save("50m_30m_Unet")

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt

# acc = history.history['accuracy']
# loss = history.history['loss']
# val_acc = history.history['val_accuracy']
# val_loss = history.history['val_loss']
# epochs = range(len(acc))

# plt.figure(figsize=(15, 6))
# plt.subplot(1, 2, 1)
# plt.plot(epochs, acc, 'b', label='Training acc',linewidth=2)
# plt.plot(epochs, val_acc, 'r--', label='Validation acc',linewidth=2)
# plt.title('Training  accuracy')
# plt.legend()

# plt.subplot(1, 2, 2)

# plt.plot(epochs, loss, 'b', label='Training loss')

# plt.plot(epochs, val_loss, 'k', label='Validationloss ')
# plt.title('Training loss')
# plt.legend()

# plt.show()

In [ ]:
# ##################################
# #IOU
# y_pred=model.predict(X_test)
# y_pred_thresholded = y_pred > 0.6

# intersection = np.logical_and(y_test, y_pred_thresholded)
# union = np.logical_or(y_test, y_pred_thresholded)
# iou_score = np.sum(intersection) / np.sum(union)
# print("IoU socre is: ", iou_score)


### Test within known data

In [ ]:
def predict_tile(tile, sens = 0.001):
    tile = tile.astype('float32') / 255.0
    test_img_other_input=np.expand_dims(tile, 0)
    prob_map = (model.predict(test_img_other_input) > sensitivty).astype(np.uint8)
    return prob_map[0]
    

file_name = "asagao_0001.png"
train_file = 'dataset/train/asagao/' + file_name
test_file = 'dataset/validation/asagao/' + file_name.replace("asagao_", "asagao_mask_" )

print(train_file, test_file)
sensitivty = 0.4
test_img_other = cv2.imread(train_file)
r = predict_tile(test_img_other, sensitivty)

plt.figure(figsize=(20,20))
actual = cv2.imread(train_file)
mask = cv2.imread(test_file)
plt.subplot(131),plt.imshow(actual),plt.title("Actual")
plt.subplot(132),plt.imshow(mask),plt.title("Actual mask")
plt.subplot(133),plt.imshow(r, cmap="gray"),plt.title("Prediction")


### Predict with tiled image

##### 1.Slice the image

In [ ]:
import os
import os
from tiled_detiled import tiled_image
import time

sensitivity = 0.6
target_f = "dataset/test/DJI_20211208095147_0040.JPG"

if len(os.listdir('dataset/test/test1')) != 0:
    print("Found previous tiled images, attemp delete")
    file_to_delete = os.listdir('dataset/test/test1')
    for f in file_to_delete:
        os.remove('dataset/test/test1/' + f)

target_im = cv2.imread(target_f)
if target_im is not None:
    tiled_image(target_im, 256, "dataset/test/test1")
else:
    print("Cannot read ifle")


#Slice image
predict_file = sorted(glob("dataset/test/test1/*.png"))
predict_file = iter(predict_file)
# print(predict_file)

predicted_list = []

#remove existing file
if len(os.listdir('dataset/test/result_test1')) != 0:
    print("Found previous predicted files, attemp delete")
    file_to_delete = os.listdir('dataset/test/result_test1')
    for f in file_to_delete:
        os.remove('dataset/test/result_test1/' + f)
#predict slided image
nn = 1
for f in predict_file:
    target = f
    # print(target)
    inputs = cv2.imread(target)
    tic = time.time()
    r = predict_tile(inputs, sensitivity)
    toc = time.time()
    print("predicted %s in %.8f s" %(f, toc - tic))
    tf.keras.preprocessing.image.save_img('dataset/test/result_test1/result_%s.png'% str(nn).zfill(4),r)
    nn += 1

#save otuput
from glob import glob
import math
result_heigth = 5460
result_width = 8192
tile_size = 256


file = sorted(glob("dataset/test/result_test1/*"))

temp = np.zeros((math.ceil(result_heigth / tile_size) * tile_size, math.ceil(result_width / tile_size) * tile_size,3))

i = 0
for r in range(0, temp.shape[0], tile_size):
    for c in range(0, temp.shape[1], tile_size):
        tile = cv2.imread(file[i])
        temp[r:r + tile_size, c:c + tile_size,:] = tile 
        i += 1

cv2.imwrite("predicted_location.jpg" , temp[:5460, :8192,:])
# cv2.imwrite("predicted_location_DJI_20211208095147_0040.JPG" , temp[:5460, :8192,:])


In [ ]:
def convert_bw(img):
    grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bw = cv2.threshold(grayImage, thresh, 255, cv2.THRESH_BINARY)[1]
    image_bw = Image.fromarray(bw)
    return image_bw

_ground_truth = cv2.imread('DJI_20211208095147_0040_mask.jpg')
_pred = cv2.imread("predicted_location_0040.jpg")

gt = convert_bw(_ground_truth)
pd = convert_bw(_pred)


intersection = np.logical_and(gt, pd)
union = np.logical_or(gt, pd)
iou_score = np.sum(intersection) / np.sum(union)
print("IoU socre is: ", iou_score)

